# Sagemaker-Huggingface SDK Training Job for RAC
In this notebook I finetune the initial version of the Rule adherance calssifier for the discord bot on the preared tokenized wikipedia comments data. The model being fine tuned is distilbert-uncased.

For context about this method of training: The sagemaker-huggingface SDK runs a training job specified in the train.py script. The model and hyperparams are specified in the huggingface estimator object. The prepared data is uploaded to an s3 bucket and the URI of each dataset is passed to estimator in the .fit() call. The base distilbert model is also upload to s3 in this way to be passed into the training job.

## Setup Sagemaker Environment

In [1]:
import sagemaker
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

In [2]:
import os

os.getcwd()

'/home/ec2-user/SageMaker/RAC_training_sagemaker'

## Huggingface Estimator
The huggingface estimator carries out the training script train.py optimized by sagemaker
The deep learning container does not have internet acces so I need to upload it to s3 and pass it into the estimator.

In [3]:
from sagemaker.huggingface import HuggingFace


# hyperparameters which are passed to the training job
hyperparameters={'epochs': 1,
                 'per_device_train_batch_size': 32,
                 'model_name_or_path': 'distilbert-base-uncased'
                 }

# create the Estimator
huggingface_estimator = HuggingFace(
        entry_point='train.py',
        source_dir='/home/ec2-user/SageMaker/RAC_training_sagemaker',
        instance_type='ml.g4dn.xlarge',
        instance_count=1,
        role=role,
        transformers_version='4.28.1',
        pytorch_version='2.0.0',
        py_version='py310',
        hyperparameters = hyperparameters,
        vpc_config={  #for using custom datasets
            'Subnets': ["get these from your VPC dash"],    
            'SecurityGroupIds': ['Ive delete sensitive info before uploading to github'] 
        }
)

## Upload datasets to s3 for estimator to access

The below code creates a new s3 bucket to store our datasets.

In [5]:
import boto3

s3 = boto3.client('s3', region_name='us-east-2')
bucket_name = 'hambart-training'

# Create a new bucket in the us-east-2 region
s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})

# Check the location of an existing bucket
response = s3.get_bucket_location(Bucket=bucket_name)
bucket_location = response['LocationConstraint']
print('Bucket location:', bucket_location)


Bucket location: us-east-2


The datasets have already been tokenized and were saved as datasets.Dataset objects in the RAC_dataset_tokenization.ipynb notebook.

Below we upload the files for each dataset into seperate directories created in the s3 bucket.

In [6]:
import os

s3 = boto3.resource('s3')
bucket_name = 'hambart-training'

base_path = '/home/ec2-user/SageMaker/RAC_training_sagemaker/datasets_dir/'
dataset_paths = [os.path.join(base_path, 'train_dataset'),
                 os.path.join(base_path, 'test_dataset'),
                 os.path.join(base_path, 'val_dataset')]


def upload_directory_to_s3(directory_path, s3_bucket, s3_key_prefix=''):
    '''
    uploads dataset folders to new dir in s3 bucket 
    for estimator to access. dir names are that which
    they are pulled from.
    '''
    directory_name = os.path.basename(directory_path)
    for root, dirs, files in os.walk(directory_path):
        for file in files:
            file_path = os.path.join(root, file)
            s3_key = os.path.join(s3_key_prefix, directory_name, os.path.relpath(file_path, directory_path))
            s3_bucket.Object(s3_key).upload_file(Filename=file_path)

bucket = s3.Bucket(bucket_name)

for folder_path in dataset_paths:
    upload_directory_to_s3(folder_path, bucket)


Below we create and print the s3 buck uris to each of the dataset dirs. These will be passed to the trainer.

In [7]:
train_dataset_uri = 's3://{}/{}'.format(bucket_name, 'train_dataset')
test_dataset_uri = 's3://{}/{}'.format(bucket_name, 'test_dataset')
val_dataset_uri = 's3://{}/{}'.format(bucket_name, 'val_dataset')

print(train_dataset_uri, '\n', test_dataset_uri, '\n', val_dataset_uri, sep='')

s3://hambart-training/train_dataset
s3://hambart-training/test_dataset
s3://hambart-training/val_dataset


## Upload base model to s3
We also need to upload the distilbert base uncased model to s3 and pass it into the training job. 

Below we download it and save it to this notebooks s3 bucket.

In [9]:
!pip install transformers

In [10]:
from transformers import DistilBertForSequenceClassification, DistilBertConfig

# I am specifying two output labels since we are fine tuning for binary sentiment analysis
num_labels = 2
config = DistilBertConfig.from_pretrained("distilbert-base-uncased", num_labels=num_labels)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", config=config)

model.save_pretrained("./distilbert-base-uncased-model") 

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Below we upload the model to the bucket

In [11]:
# The name of the directory to be created in the bucket
directory_name = "distilbert-base-uncased-model"

# Upload the entire directory
for root, dirs, files in os.walk("./distilbert-base-uncased-model"):
    for filename in files:
        local_path = os.path.join(root, filename)
        s3_path = os.path.join(directory_name, os.path.relpath(local_path, "./distilbert-base-uncased-model"))
        
        bucket.upload_file(local_path, s3_path)


## Fit Data

Calling the .fit() method on the hugging face estimator will carry out the training specified in train.py

In [12]:
huggingface_estimator.fit({'train': train_dataset_uri, 'test': test_dataset_uri})

Using provided s3_resource


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2023-08-14-18-03-20-698


2023-08-14 18:04:42 Starting - Starting the training job...
2023-08-14 18:04:57 Starting - Preparing the instances for training......
2023-08-14 18:05:55 Downloading - Downloading input data...
2023-08-14 18:06:25 Training - Downloading the training image.................................
2023-08-14 18:11:47 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-08-14 18:11:59,623 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-08-14 18:11:59,642 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-08-14 18:11:59,651 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-08-14 18:11:59,659 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-08-14 18:12:07,175 sagemaker-training-toolkit INFO     No Neurons

2023-08-14 18:12:10,882 - __main__ - INFO -  loaded train_dataset length is: 110000
2023-08-14 18:12:10,882 - __main__ - INFO -  loaded test_dataset length is: 25000
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
5%|▍         | 159/3438 [00:46<16:01,  3.41it/s]


9%|▉         | 311/3438 [01:31<15:39,  3.33it/s]


...

72%|███████▏  | 282/391 [01:04<00:25,  4.34it/s]#033[A


100%|██████████| 391/391 [01:29<00:00,  4.97it/s]#033[A
#033[A
{'eval_loss': 0.08413369953632355, 'eval_accuracy': 0.96832, 'eval_f1': 0.8383013474887708, 'eval_precision': 0.8807378807378807, 'eval_recall': 0.7997662641215426, 'eval_runtime': 89.8294, 'eval_samples_per_second': 278.305, 'eval_steps_per_second': 4.353, 'epoch': 1.0}
#015100%|██████████| 391/391 [01:29<00:00,  4.97it/s]#033[A
#033[A
{'train_runtime': 1182.0168, 'train_samples_per_second': 93.061, 'train_steps_per_second': 2.909, 'train_loss': 0.11198443157025172, 'epoch': 1.0}
6%|▌         | 22/391 [00:04<01:24,  4.37it/s]


56%|█████▌    | 218/391 [00:49<00:39,  4.36it/s]


100%|██████████| 391/391 [01:29<00:00,  4.38it/s]
***** Eval results *****



2023-08-14 18:33:32 Uploading - Uploading generated training model2023-08-14 18:33:26,806 sagemaker-training-toolkit INFO     Waiting for the process to finish and give a return code.
2023-08-14 18:33:26,806 sagemaker-training-toolkit INFO     Done waiting for a return code. Received 0 from exiting process.
2023-08-14 18:33:26,807 sagemaker-training-toolkit INFO     Reporting training SUCCESS

2023-08-14 18:37:58 Completed - Training job completed
Training seconds: 1923
Billable seconds: 1923
